В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает [данные](https://stepik.org/media/attachments/lesson/387691/cars.csv). 

### Задачи:
1. Загрузить данные, проверить правильность, наличие пропущенных значений, типы данных.  
2. Создать новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправить названия и проверить изменения.  
3. Оставить только часть предикторов, после чего посчитать корреляцию между price и другими переменными.  
4. Преобразовать категориальные переменные с помощью pd.get_dummies().  
5. Построить модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? ($R^2$)  
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в $R^2$, коэффициентах и их значимости. Какую модель лучше оставить?   
7. Заполните пропуски в результатах.  

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm 

In [2]:
# считываем данные
cars = pd.read_csv('https://stepik.org/media/attachments/lesson/387691/cars.csv')

In [3]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
cars.shape

(205, 26)

In [5]:
# проверим пропуски
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [6]:
# количество уникальных марок автомобилей
cars.CarName.nunique()

147

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используя столбец CarName, разобъём значения ячеек по пробелу и запишем в колонку первый элемент.

In [7]:
# добавим колонку
cars['company'] = cars.CarName.apply(lambda x: x.split()[0])

In [8]:
# удалим неиспользуемые колонки
cars.drop(['CarName', 'car_ID'], axis=1, inplace=True)

In [9]:
# количество уникальных названий автомобилей после преобразования
cars.company.nunique()

28

In [10]:
# уникальные названия автомобилей
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

Теперь внимательнее посмотрим на уникальные значения company. Часть из них оказалась с ошибками!

In [11]:
# словарь корректировок марок автомобилей
mistake_dict  = {'toyouta': 'toyota',
                'maxda': 'mazda',
                'Nissan': 'nissan',
                'porcshce': 'porsche',
                'vokswagen': 'volkswagen',
                'vw': 'volkswagen'}

In [12]:
# функция замены значений колонки по словарю
def replacer(x, dic):
    if x in dic:
        s = x.replace(x, dic[x])
        return s
    return x

In [13]:
# исправляем ошибки в названиях автомобилей
cars['company'] = cars.company.apply(lambda z: replacer(z, mistake_dict))

In [14]:
cars.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [15]:
# количество уникальных названий автомобилей после исправления ошибок
cars.company.nunique()

22

In [16]:
# уникальные названия автомобилей
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

Предсказывать будем 'price'.  
Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

In [17]:
column_list = ['price', 'company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength' ,'carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower']

Посчитаем корреляцию между `price` и другими переменными.

In [18]:
cars[column_list].corr().round(2)

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.00,0.58,0.68,0.76,0.84,0.87,0.55,0.81
wheelbase,0.58,1.00,0.87,0.80,0.78,0.57,0.49,0.35
carlength,0.68,0.87,1.00,0.84,0.88,0.68,0.61,0.55
carwidth,0.76,0.80,0.84,1.00,0.87,0.74,0.56,0.64
curbweight,0.84,0.78,0.88,0.87,1.00,0.85,0.65,0.75
enginesize,0.87,0.57,0.68,0.74,0.85,1.00,0.58,0.81
boreratio,0.55,0.49,0.61,0.56,0.65,0.58,1.00,0.57
horsepower,0.81,0.35,0.55,0.64,0.75,0.81,0.57,1.00


In [19]:
# найдём категориальные переменные
cars[column_list].dtypes

price             float64
company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
dtype: object

In [20]:
# имена колонок имеющих формат object
object_columns = ['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber']

# имена колонок имеющих числовой формат 
digital_columns = (['price', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower'])

In [21]:
df_dummy = pd.get_dummies(data=cars[object_columns], drop_first = True)

In [22]:
total_df = pd.concat((cars[digital_columns], df_dummy), axis=1)

In [23]:
total_df.shape

(205, 49)

Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

In [24]:
results = smf.ols('price ~ horsepower', total_df).fit() 

In [25]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           1.48e-48
Time:                        08:24:55   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.000   -5555.163   -1888.360
horsepower   163.2631      8.351     19.549      0.000     146.796     179.730
==============================================================================
Omnibus:                       47.741   Durbin-Watson:                   0.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.702
Skew:                           1.141   Prob(JB):                     1.22e-20
Kurtosis:                       5.352   Cond. No.                         314.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Теперь – две модели:
- модель со всеми предикторами
- модель со всеми предикторами, кроме марок машин

In [26]:
# модель со всеми предикторами
X = sm.add_constant(total_df.drop('price', axis=1))
model = sm.OLS(total_df.price, X)
results_2 = model.fit()
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           4.86e-89
Time:                        08:29:33   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [27]:
# модель со всеми предикторами, кроме марок машин
x_df = total_df[['wheelbase', 'carlength', 'carwidth', 'curbweight',
       'enginesize', 'boreratio', 'horsepower', 'fueltype_gas',
       'aspiration_turbo', 'carbody_hardtop', 'carbody_hatchback',
       'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor', 'cylindernumber_five',
       'cylindernumber_four', 'cylindernumber_six', 'cylindernumber_three',
       'cylindernumber_twelve', 'cylindernumber_two']]

X = sm.add_constant(x_df)
model = sm.OLS(total_df.price, X)
results_3 = model.fit()
print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           9.86e-81
Time:                        08:35:09   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

Обратим внимание на изменения в $R^2$, коэффициентах и их значимости.  
Оставляем вариант, где меньше предикторов, ведь Adj. R-squared изменился не очень сильно, а часть марок вообще не значима.

Резюмируем.  
Выбранная модель объясняет примерно 90% дисперсии (Adj. R-squared: 0.901). Среди предикторов 10 из 27 оказались не значимыми (p > 0.05).